In [1]:
import sqlalchemy

In [2]:
sqlalchemy.__version__ 

'1.1.13'

In [3]:
from sqlalchemy import create_engine

In [4]:
engine = create_engine('sqlite:////data/sqlite.db', echo=True)

In [5]:
from sqlalchemy.ext.declarative import declarative_base

In [6]:
Base = declarative_base()

In [7]:
Base

sqlalchemy.ext.declarative.api.Base

In [8]:
engine

Engine(sqlite:////data/sqlite.db)

In [9]:
from sqlalchemy import Column, Integer, String, Enum

In [10]:
class Winner(Base):
    __tablename__ = 'winners'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    catagory = Column(String)
    year = Column(Integer)
    nationality = Column(String)
    sex = Column(Enum('male', 'female'))

    def __repr__(self):
        return "<Winner(name='%s', catagory='%s', year='%s')>" % (self.name, self.catagory, self.year)


In [11]:
Base.metadata.create_all(engine)

2017-08-29 19:06:05,551 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-08-29 19:06:05,553 INFO sqlalchemy.engine.base.Engine ()
2017-08-29 19:06:05,555 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-08-29 19:06:05,556 INFO sqlalchemy.engine.base.Engine ()
2017-08-29 19:06:05,559 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("winners")
2017-08-29 19:06:05,561 INFO sqlalchemy.engine.base.Engine ()
2017-08-29 19:06:05,565 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE winners (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	catagory VARCHAR, 
	year INTEGER, 
	nationality VARCHAR, 
	sex VARCHAR(6), 
	PRIMARY KEY (id), 
	CHECK (sex IN ('male', 'female'))
)


2017-08-29 19:06:05,567 INFO sqlalchemy.engine.base.Engine ()
2017-08-29 19:06:05,577 INFO sqlalchemy.engine.base.Engine COMMIT


In [12]:
from sqlalchemy.orm import sessionmaker 

In [13]:
Session = sessionmaker(bind=engine)

In [14]:
session = Session()

In [15]:
nobel_winners = [
    {'catagory': 'Physics',
     'name': 'Albert Einstein',
     'nationality': 'Swiss',
     'sex': 'male',
     'year': 1921},
    {'catagory': 'Physics',
     'name': 'Paul Dirac',
     'nationality': 'British',
     'sex': 'male',
     'year': 1933},
    {'catagory': 'Chemistry',
     'name': 'Marie Curie',
     'nationality': 'Polish',
     'sex': 'female',
     'year': 1911},
]

In [16]:
albert = Winner(**nobel_winners[0])

In [17]:
albert

<Winner(name='Albert Einstein', catagory='Physics', year='1921')>

In [18]:
session.add(albert)

In [19]:
session.new

IdentitySet([<Winner(name='Albert Einstein', catagory='Physics', year='1921')>])

In [20]:
session.add(Winner(**nobel_winners[1]))

In [21]:
session.new

IdentitySet([<Winner(name='Albert Einstein', catagory='Physics', year='1921')>, <Winner(name='Paul Dirac', catagory='Physics', year='1933')>])

In [22]:
session.expunge(albert)

In [23]:
session.new

IdentitySet([<Winner(name='Paul Dirac', catagory='Physics', year='1933')>])

In [24]:
session.commit()

2017-08-29 19:06:05,726 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-29 19:06:05,729 INFO sqlalchemy.engine.base.Engine INSERT INTO winners (name, catagory, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2017-08-29 19:06:05,730 INFO sqlalchemy.engine.base.Engine ('Paul Dirac', 'Physics', 1933, 'British', 'male')
2017-08-29 19:06:05,734 INFO sqlalchemy.engine.base.Engine COMMIT


In [25]:
winner_rows = [Winner(**w) for w in nobel_winners]

In [26]:
winner_rows

[<Winner(name='Albert Einstein', catagory='Physics', year='1921')>,
 <Winner(name='Paul Dirac', catagory='Physics', year='1933')>,
 <Winner(name='Marie Curie', catagory='Chemistry', year='1911')>]

In [27]:
session.add_all(winner_rows)

In [28]:
session.commit()

2017-08-29 19:06:05,779 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-29 19:06:05,784 INFO sqlalchemy.engine.base.Engine INSERT INTO winners (name, catagory, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2017-08-29 19:06:05,786 INFO sqlalchemy.engine.base.Engine ('Albert Einstein', 'Physics', 1921, 'Swiss', 'male')
2017-08-29 19:06:05,790 INFO sqlalchemy.engine.base.Engine INSERT INTO winners (name, catagory, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2017-08-29 19:06:05,793 INFO sqlalchemy.engine.base.Engine ('Paul Dirac', 'Physics', 1933, 'British', 'male')
2017-08-29 19:06:05,795 INFO sqlalchemy.engine.base.Engine INSERT INTO winners (name, catagory, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2017-08-29 19:06:05,797 INFO sqlalchemy.engine.base.Engine ('Marie Curie', 'Chemistry', 1911, 'Polish', 'female')
2017-08-29 19:06:05,799 INFO sqlalchemy.engine.base.Engine COMMIT


In [29]:
session.query(Winner).count()

2017-08-29 19:06:05,813 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-29 19:06:05,816 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT winners.id AS winners_id, winners.name AS winners_name, winners.catagory AS winners_catagory, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners) AS anon_1
2017-08-29 19:06:05,817 INFO sqlalchemy.engine.base.Engine ()


4

In [30]:
session.query(Winner)

In [31]:
list(session.query(Winner))

2017-08-29 19:06:05,841 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.catagory AS winners_catagory, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners
2017-08-29 19:06:05,844 INFO sqlalchemy.engine.base.Engine ()


[<Winner(name='Paul Dirac', catagory='Physics', year='1933')>,
 <Winner(name='Albert Einstein', catagory='Physics', year='1921')>,
 <Winner(name='Paul Dirac', catagory='Physics', year='1933')>,
 <Winner(name='Marie Curie', catagory='Chemistry', year='1911')>]

In [32]:
result = session.query(Winner).filter_by(catagory='Physics')

In [33]:
list(result)

2017-08-29 19:06:05,864 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.catagory AS winners_catagory, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.catagory = ?
2017-08-29 19:06:05,866 INFO sqlalchemy.engine.base.Engine ('Physics',)


[<Winner(name='Paul Dirac', catagory='Physics', year='1933')>,
 <Winner(name='Albert Einstein', catagory='Physics', year='1921')>,
 <Winner(name='Paul Dirac', catagory='Physics', year='1933')>]

In [34]:
res = session.query(Winner).order_by('year')

In [35]:
list(res)

2017-08-29 19:06:05,888 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.catagory AS winners_catagory, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners ORDER BY winners.year
2017-08-29 19:06:05,892 INFO sqlalchemy.engine.base.Engine ()


[<Winner(name='Marie Curie', catagory='Chemistry', year='1911')>,
 <Winner(name='Albert Einstein', catagory='Physics', year='1921')>,
 <Winner(name='Paul Dirac', catagory='Physics', year='1933')>,
 <Winner(name='Paul Dirac', catagory='Physics', year='1933')>]

In [36]:
def inst_to_dict(inst, delete_id=True):
    dat = {}
    for column in inst.__table__.columns:
        dat[column.name] = getattr(inst, column.name)
    if delete_id:
        dat.pop('id')
    return dat

In [37]:
winner_rows = session.query(Winner)
nobel_winners_x = [inst_to_dict(w, False) for w in winner_rows]

2017-08-29 19:06:05,916 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.catagory AS winners_catagory, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners
2017-08-29 19:06:05,918 INFO sqlalchemy.engine.base.Engine ()


In [38]:
nobel_winners

[{'catagory': 'Physics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921},
 {'catagory': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'catagory': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

In [39]:
nobel_winners_x

[{'catagory': 'Physics',
  'id': 1,
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'catagory': 'Physics',
  'id': 2,
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921},
 {'catagory': 'Physics',
  'id': 3,
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'catagory': 'Chemistry',
  'id': 4,
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

In [40]:
session.query(Winner).filter_by(id=3).delete()

2017-08-29 19:06:05,953 INFO sqlalchemy.engine.base.Engine DELETE FROM winners WHERE winners.id = ?
2017-08-29 19:06:05,956 INFO sqlalchemy.engine.base.Engine (3,)


1

In [41]:
winner_rows = session.query(Winner)
nobel_winners_x = [inst_to_dict(w, False) for w in winner_rows]
nobel_winners_x

2017-08-29 19:06:05,972 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.catagory AS winners_catagory, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners
2017-08-29 19:06:05,976 INFO sqlalchemy.engine.base.Engine ()


[{'catagory': 'Physics',
  'id': 1,
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'catagory': 'Physics',
  'id': 2,
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921},
 {'catagory': 'Chemistry',
  'id': 4,
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

In [42]:
marie = session.query(Winner).get(4)

In [43]:
marie

<Winner(name='Marie Curie', catagory='Chemistry', year='1911')>

In [44]:
marie.nationality = 'French'

In [45]:
session.dirty

IdentitySet([<Winner(name='Marie Curie', catagory='Chemistry', year='1911')>])

In [46]:
session.new

IdentitySet([])

In [47]:
session.commit()

2017-08-29 19:06:06,036 INFO sqlalchemy.engine.base.Engine UPDATE winners SET nationality=? WHERE winners.id = ?
2017-08-29 19:06:06,039 INFO sqlalchemy.engine.base.Engine ('French', 4)
2017-08-29 19:06:06,041 INFO sqlalchemy.engine.base.Engine COMMIT


In [48]:
session.dirty

IdentitySet([])

In [49]:
session.query(Winner).get(4).nationality

2017-08-29 19:06:06,063 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-29 19:06:06,066 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.catagory AS winners_catagory, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.id = ?
2017-08-29 19:06:06,068 INFO sqlalchemy.engine.base.Engine (4,)


'French'

In [50]:
list(session.query(Winner))

2017-08-29 19:06:06,080 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.catagory AS winners_catagory, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners
2017-08-29 19:06:06,082 INFO sqlalchemy.engine.base.Engine ()


[<Winner(name='Paul Dirac', catagory='Physics', year='1933')>,
 <Winner(name='Albert Einstein', catagory='Physics', year='1921')>,
 <Winner(name='Marie Curie', catagory='Chemistry', year='1911')>]

In [51]:
# Winner.__table__.drop(engine)